In [14]:
import itertools
import mdtraj as md
import mdtraj.testing



In [15]:
t = md.load('./data/pdbs/AF-C0H3V2-F1.pdb')
print(t)
hbonds = md.baker_hubbard(t)
for hbond in hbonds:
    print(hbond)

<mdtraj.Trajectory with 1 frames, 1105 atoms, 143 residues, without unitcells>


C:\Users\marik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mdtraj\formats\pdb\pdbfile.py:206: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn(


In [16]:
label = lambda hbond: f"{t.topology.atom(hbond[0])} -- {t.topology.atom(hbond[2])}"

for hbond in hbonds:
    print(label(hbond))

In [16]:
import os
import numpy as np
path = './data'
N_H_bond = float('1.0752') # DOI: 10.1007/s00894-015-2587-3
O_H_bond = float('1.63') # DOI: 10.4236/jbpc.2020.112002
Donor_list = [('GLN', 'NE2', 'HE21'), ('GLN', 'NE2', 'HE22'), ('GLU', 'OE2', 'HE2'), ('ASP', 'OD2', 'HD2'), ('ASN', 'ND2', 'HD21'), ('ASN', 'ND2', 'HD22'),
                ('HIS', 'NE2', 'HE2'), ('HIS', 'ND1', 'HD2'), ('LYS', 'NZ', 'HZ1'), ('LYS', 'NZ', 'HZ2'), ('LYS', 'NZ', 'HZ3'), ('ARG', 'NE', 'HE'), 
                ('ARG', 'NH1', 'HH11'), ('ARG', 'NH1', 'HH12'), ('ARG', 'NH2', 'HH21'), ('ARG', 'NH2', 'HH22'), ('SER', 'OG', 'HG'), ('THR', 'OG1', 'HG1'), 
                ('TRP', 'NE1', 'HE1'), ('TYR', 'OH', 'HH')]
Acceptor_list = [('GLN', 'OE1'), ('ASP', 'OD1'), ('ASP', 'OD2'), ('ASN', 'OD1')]

with open(os.path.join(path, 'C0H3Z2.pqr')) as f:
    Donor_array = np.empty((0, 4))
    H_array = np.empty((0, 4))
    Acceptor_array = np.empty((0, 4))
    aa_cache = []
    atom_cache = []
    test_count = 0
    for line in f:
        if line.startswith('ATOM'):
            if not aa_cache:
                aa_cache.append(line.split()[3])
                aa_cache.append(line.split()[4])
                atom_cache.append(line)
            elif aa_cache[1] == line.split()[4]:
                atom_cache.append(line)
            elif aa_cache[1] != line.split()[4]:
                test_count += 1             
                for n in range(len(Donor_list)):
                    if aa_cache[0] == Donor_list[n][0]:
                        for i in range(len(atom_cache)):
                            if Donor_list[n][1] == atom_cache[i].split()[2]:
                                if ('GLU' == aa_cache[0]) and 'OE2' in atom_cache[i].split()[2]:
                                    if any('HE2' in string for string in atom_cache):
                                        line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                        line_array = line_array.astype('float64')
                                        Donor_array = np.append(Donor_array, line_array, axis=0) 
                                elif ('ASP' == aa_cache[0]) and 'OD2' in atom_cache[i].split()[2]:
                                    if any('HD2' in string for string in atom_cache):
                                        line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                        line_array = line_array.astype('float64')
                                        Donor_array = np.append(Donor_array, line_array, axis=0)                                             
                                elif ('HIS' == aa_cache[0]) and 'ND1' in atom_cache[i].split()[2]:
                                    if any ('HD1' in string for string in atom_cache):
                                        line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                        line_array = line_array.astype('float64')
                                        Donor_array = np.append(Donor_array, line_array, axis=0)
                                elif ('HIS' == aa_cache[0]) and 'NE2' in atom_cache[i].split()[2]:
                                    if any('HE2' in string for string in atom_cache):
                                        line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                        line_array = line_array.astype('float64')
                                        Donor_array = np.append(Donor_array, line_array, axis=0)
                                else: 
                                    line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                    line_array = line_array.astype('float64')
                                    Donor_array = np.append(Donor_array, line_array, axis=0)         
                            elif Donor_list[n][2] == atom_cache[i].split()[2]:
                                line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                line_array = line_array.astype('float64')
                                H_array = np.append(H_array, line_array, axis=0)
                for n in range(len(Acceptor_list)):
                    if aa_cache[0] == Acceptor_list[n][0]:
                        for i in range(len(atom_cache)):
                            if Acceptor_list[n][1] == atom_cache[i].split()[2]:
                                if ('GLU' == aa_cache[0]) and 'OE2' in atom_cache[i].split()[2]:
                                    if any('HE2' not in string for string in atom_cache):
                                        line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                        line_array = line_array.astype('float64')
                                        Acceptor_array = np.append(Acceptor_array, line_array, axis=0)
                                elif ('ASP' == aa_cache[0]) and 'OD' in atom_cache[i].split()[2]:
                                    if any('HD' not in string for string in atom_cache):
                                        line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                        line_array = line_array.astype('float64')
                                        Acceptor_array = np.append(Acceptor_array, line_array, axis=0)
                                else: 
                                    line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                    line_array = line_array.astype('float64')
                                    Acceptor_array = np.append(Acceptor_array, line_array, axis=0) 

 
                                        
                aa_cache = []
                atom_cache = []
    print(Donor_array.shape)
    print(H_array.shape)
    print(Acceptor_array.shape)
    print(Donor_array[:,0])
    print(H_array[:,0])


(63, 4)
(63, 4)
(11, 4)
[  28.   28.   28.   67.  136.  137.  137.  138.  138.  208.  220.  252.
  253.  253.  254.  254.  276.  277.  277.  278.  278.  300.  301.  301.
  302.  302.  339.  339.  339.  390.  408.  408.  431.  431.  484.  512.
  541.  541.  541.  562.  563.  563.  564.  564.  596.  617.  618.  618.
  619.  619.  683.  683.  755.  809.  867.  993.  994.  994.  995.  995.
 1055. 1055. 1055.]
[  39.   40.   41.   77.  148.  149.  150.  151.  152.  213.  227.  264.
  265.  266.  267.  268.  288.  289.  290.  291.  292.  312.  313.  314.
  315.  316.  350.  351.  352.  400.  416.  417.  433.  434.  488.  522.
  552.  553.  554.  574.  575.  576.  577.  578.  605.  629.  630.  631.
  632.  633.  685.  686.  760.  819.  877. 1005. 1006. 1007. 1008. 1009.
 1066. 1067. 1068.]


In [92]:
Donor_list = [('GLN', 'NE2', 'HE22', 'HE23'), ('GLU', 'OE2', 'HE2'), ('ASP', 'OD2', 'HD2'), ('ASN', 'ND2', 'HD21', 'HD22'), 
                  ('HIS', 'NE2', 'HE2'), ('HIS', 'ND1', 'ND2'), ('LYS', 'NZ', 'HZ1', 'HZ2', 'HZ3'), ('ARG', 'NE', 'HE'), 
                  ('ARG', 'NH1', 'HH11', 'HH12'), ('ARG', 'NH2', 'HH21', 'HH22'), ('SER', 'OG', 'HG'), ('THR', 'OG1', 'HG1'), 
                  ('TRP', 'NE1', 'HE1'), ('TYR', 'OH', 'HH')]
for i in range(len(Donor_list[3])-2):
    print(i)


0
1


In [7]:
string = 'ATOM     12  HB3 MET     1     -20.987   0.463   6.535  0.0000 0.0000'
string2 ='ATOM      5  O   MET     1     -18.415  -1.135   7.336 -0.5500 1.4000'
list2 = [string.split()[4]]
print(list2)

if list2[0] == string2.split()[4]:
    print('yes')
else:
    print('no')


['1']
yes
